In [1]:
import numpy as np
import importlib
import operator
import util as uti
import matplotlib.pyplot as plt
from collections import OrderedDict
importlib.reload(uti)

# import math lib
import math

# import Qiskit
from qiskit import Aer, IBMQ, execute, assemble, transpile
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit.circuit.library import GroverOperator
from qiskit_machine_learning.circuit.library import RawFeatureVector

# import basic plot tools
#import qiskit.visualization as vis
from qiskit.visualization import plot_histogram
%matplotlib inline

import sys
import os
sys.path.append(os.path.abspath('../quantum'))
import oracles as ora
import grover as gro
importlib.reload(gro)

/home/kinga/.local/lib/python3.6/site-packages/qiskit/__init__.py:66: DeprecationWarning: Using Qiskit with Python 3.6 is deprecated as of qiskit-terra 0.17.0. Support for running Qiskit with Python 3.6 will be removed in qiskit-terra 0.20.0.
  DeprecationWarning,


<module 'grover' from '/home/kinga/mnt6/laspaclu/quantum/grover.py'>

In [2]:
def measure_by_prob(counts, n):
    # sort indices by prob
    sorted_counts = OrderedDict(sorted(counts.items(), key=lambda x: x[1], reverse=True))
    # take most probable value (if it is in the limits of array)
    for k in sorted_counts.keys():
        if int(k,2) < n:
            return int(k,2)
    print('no valid index found')
    return -1

In [3]:
def duerr_hoyer_minimization(distances):
    
    nn = int(math.floor(math.log2(len(distances)) + 1))
    iter_n = math.ceil(np.sqrt(2**nn)) # suggested in duerr & hoyer 
    
    threshold_oracles = ora.create_threshold_oracle_set(distances)
    
    # init first random threshold
    idx = np.random.randint(0, len(distances)-1)
    
    # iterate
    for _ in range(iter_n):
        
        # pick next threshold
        threshold = distances[idx]
        print('next minimum guess: dist[{}] = {}'.format(idx, threshold))
        marked_n = len(ora.get_indices_to_mark(distances, threshold))
        
        # create oracle combi and grover algo
        oracle_qc = ora.create_oracle_lincombi(threshold, distances, threshold_oracles)
        grover_qc = gro.grover_circuit(nn, oracle_qc, marked_n)
        
        # apply grover algo (only one shot to get a true collapsed value)
        simulator = Aer.get_backend('qasm_simulator')
        counts = execute(grover_qc, backend=simulator, shots=1000).result().get_counts(grover_qc)
        idx = measure_by_prob(counts, len(distances))
        
    print('final minimum found dist[{}] = {}'.format(idx, distances[idx]))
    return distances[idx]

In [4]:
distances = np.array([17.1,4.6,6,2,8,32,7,16,4.7,9.234,1.1,34.5,3.875, 0.3])

In [5]:
duerr_hoyer_minimization(distances)

next minimum guess: dist[2] = 6.0


/home/kinga/.local/lib/python3.6/site-packages/scipy/signal/waveforms.py:675: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  elif idx == 'mid':


next minimum guess: dist[3] = 2.0
next minimum guess: dist[13] = 0.3


/home/kinga/.local/lib/python3.6/site-packages/scipy/signal/waveforms.py:675: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  elif idx == 'mid':


next minimum guess: dist[13] = 0.3
final minimum found dist[13] = 0.3


0.3

In [14]:
a = np.empty((2,3))

In [15]:
a

array([[0., 0., 0.],
       [0., 0., 0.]])

In [2]:
rng = np.random.RandomState(2)

In [4]:
i = rng.permutation(10)[:2]

In [5]:
i

array([1, 6])

In [6]:
rng.permutation(10)

array([4, 9, 5, 0, 8, 1, 2, 6, 7, 3])

In [11]:
idx = np.random.choice(10, size=3, replace=False)

In [12]:
a = np.random.randint(20, size=(10,4))

In [13]:
a[idx]

array([[ 3,  1, 18,  7],
       [19,  4,  1, 18],
       [18, 12, 19, 14]])

In [14]:
a

array([[ 4, 12, 16, 10],
       [13, 17, 14, 15],
       [13, 13, 13,  5],
       [19,  4,  1, 18],
       [16,  5, 19, 12],
       [17, 14, 15, 14],
       [18, 12, 19, 14],
       [ 1,  6,  2,  2],
       [15, 17,  9,  2],
       [ 3,  1, 18,  7]])

In [19]:
assign = np.random.choice(3, size=10)

In [20]:
assign

array([0, 0, 2, 0, 0, 1, 1, 0, 0, 2])

In [23]:
a[assign == 1]

array([[17, 14, 15, 14],
       [18, 12, 19, 14]])

In [25]:
a[assign == 1].mean(axis=0)

array([17.5, 13. , 17. , 14. ])